# Session 9: Synthetic Data Generation and RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow, and use it to evaluate and iterate on a RAG pipeline with LangSmith!

**Learning Objectives:**
- Understand Ragas' knowledge graph-based synthetic data generation workflow
- Generate synthetic test sets with different query synthesizer types
- Load synthetic data into LangSmith for evaluation
- Evaluate a RAG chain using LangSmith evaluators
- Iterate on RAG pipeline parameters and measure the impact

## Table of Contents:

- **Breakout Room #1:** Synthetic Data Generation with Ragas
  - Task 1: Dependencies and API Keys
  - Task 2: Data Preparation and Knowledge Graph Construction
  - Task 3: Generating Synthetic Test Data
  - Question #1 & Question #2
  - 🏗️ Activity #1: Custom Query Distribution

- **Breakout Room #2:** RAG Evaluation with LangSmith
  - Task 4: LangSmith Dataset Setup
  - Task 5: Building a Basic RAG Chain
  - Task 6: Evaluating with LangSmith
  - Task 7: Modifying the Pipeline and Re-Evaluating
  - Question #3 & Question #4
  - 🏗️ Activity #2: Analyze Evaluation Results

---
# 🤝 Breakout Room #1
## Synthetic Data Generation with Ragas

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/dereky/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/dereky/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# import os
# import getpass

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")


# Load environment variables from .env file
from uuid import uuid4
import getpass
import os
from dotenv import load_dotenv
load_dotenv()

def get_api_key(env_var: str, prompt: str) -> str:
    """Get API key from environment or prompt user."""
    value = os.environ.get(env_var, "")
    if not value:
        value = getpass.getpass(prompt)
        if value:
            os.environ[env_var] = value
    return value

cohere_key = get_api_key("COHERE_API_KEY", "Cohere API Key: ")
if cohere_key:
    print("cohere API key set")
else:
    print("Warning: No Cohere API key configured")

# Set Anthropic API Key (default for Deep Agents)
anthropic_key = get_api_key("ANTHROPIC_API_KEY", "Anthropic API Key: ")
if anthropic_key:
    print("Anthropic API key set")
else:
    print("Warning: No Anthropic API key configured")

# Optional: OpenAI for alternative models and subagents
openai_key = get_api_key("OPENAI_API_KEY", "OpenAI API Key (press Enter to skip): ")
if openai_key:
    print("OpenAI API key set")
else:
    print("OpenAI API key not configured (optional)")

# Optional: LangSmith for tracing
langsmith_key = get_api_key("LANGCHAIN_API_KEY", "LangSmith API Key (press Enter to skip): ")

if langsmith_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Deep Agents - {uuid4().hex[0:8]}"
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")
else:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")

cohere API key set
Anthropic API key set
OpenAI API key set
LangSmith tracing enabled. Project: AIE9 - Deep Agents - d7ae3586


We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

#os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
# os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data using two complementary guides — a Health & Wellness Guide covering exercise, nutrition, sleep, and stress management, and a Mental Health & Psychology Handbook covering mental health conditions, therapeutic approaches, resilience, and daily mental health practices. The topical overlap between documents helps RAGAS build rich cross-document relationships in the knowledge graph.

Next, let's load our data into a familiar LangChain format using the `TextLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader("data/", glob="*.txt", loader_cls=TextLoader)
docs = loader.load()
print(f"Loaded {len(docs)} documents: {[d.metadata['source'] for d in docs]}")

Loaded 2 documents: ['data/MentalHealthGuide.txt', 'data/HealthWellnessGuide.txt']


### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [8]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [14]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Property 'headlines' already exists in node 'b9a254'. Skipping!
Property 'headlines' already exists in node 'e3874c'. Skipping!


Applying HeadlineSplitter:   0%|          | 0/8 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

...

Applying CustomNodeFilter:   0%|          | 0/13 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node 'e3874c'. Skipping!
Property 'summary_embedding' already exists in node 'b9a254'. Skipping!
Property 'entities' already exists in node '0115e6'. Skipping!
Property 'entities' already exists in node '784e79'. Skipping!
Property 'themes' already exists in node '773de5'. Skipping!
Property 'entities' already exists in node 'c1d82d'. Skipping!
Property 'themes' already exists in node '784e79'. Skipping!
Property 'themes' already exists in node '0115e6'. Skipping!
Property 'themes' already exists in node '0520c0'. Skipping!
Property 'themes' already exists in node '934a13'. Skipping!
Property 'entities' already exists in node '773de5'. Skipping!
Property 'entities' already exists in node '934a13'. Skipping!
Property 'entities' already exists in node '0520c0'. Skipping!
Property 'themes' already exists in node 'c1d82d'. Skipping!


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 14, relationships: 28)

We can save and load our knowledge graphs as follows.

In [15]:
kg.save("usecase_data_kg.json")
usecase_data_kg = KnowledgeGraph.load("usecase_data_kg.json")
usecase_data_kg

KnowledgeGraph(nodes: 14, relationships: 28)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=usecase_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [30]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

## ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

#### Answer:
##### SingleHopSpecificQuerySynthesizer
Generate Data based on the source data in factual terms, don't try to extract any concepts, just facts and figures.  You should be able to answer from one specific chunk/document.  Use a single "hop" so don't attempt to address two topics at once, stick to a single topic.  An example would be: Why did George Washington Cross the Delaware?.

##### MultiHopSpecificQuerySynthesizer
Generate Data based on the source data in factual terms, don't try to extract any concepts, just facts and figures.  You should be have to answer from multiple specific chunks/documents. Use a multiple "hops" so DO attempt to address two topics at once, make it more complex .  An example would be: Why did George Washington Cross the Delaware, what were the British forces on the other side and how well equipped were they?.

##### MultiHopAbstractQuerySynthesizer
Generate Data based on the source data and do some analysis to extract meaning, DO try to extract concepts, do analysis, and get into conceptual deatils.  You should be have to answer from multiple specific chunks/documents.  Use a multiple "hops" so DO attempt to address two topics at once, make it more complex .  An example would be: Why did George Washington Cross the Delaware, what were the British forces on the other side?  How did this impact the Colonial Morale, supply levels and did it turn the tide of the American Revolution?



Finally, we can use our `TestSetGenerator` to generate our testset!

In [31]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,World Health Organization mental health what d...,[The Mental Health and Psychology Handbook A P...,"According to the context, the World Health Org...",single_hop_specifc_query_synthesizer
1,What is Cognitive Behavioral Therapy and how d...,[PART 2: THERAPEUTIC APPROACHES Chapter 4: Cog...,Cognitive Behavioral Therapy is one of the mos...,single_hop_specifc_query_synthesizer
2,What is CBT-I and how does it relate to mental...,[Write letters to or from your future self Jou...,CBT-I is the recommended first-line treatment ...,single_hop_specifc_query_synthesizer
3,What are some mental health resources availabl...,[social interactions How to set and maintain b...,The context mentions access to mental health r...,single_hop_specifc_query_synthesizer
4,What role do carbohydrates play in a balanced ...,[PART 2: NUTRITION AND DIET Chapter 4: Fundame...,Carbohydrates are the primary energy source in...,single_hop_specifc_query_synthesizer
5,How does drinking water and exercise affect mo...,[<1-hop>\n\nPART 2: NUTRITION AND DIET Chapter...,Drinking water is essential for bodily functio...,multi_hop_abstract_query_synthesizer
6,How can incorporating self-reflection through ...,[<1-hop>\n\nWrite letters to or from your futu...,"Incorporating journaling practices, such as wr...",multi_hop_abstract_query_synthesizer
7,How do healthy eating principles and key nutri...,[<1-hop>\n\nPART 2: NUTRITION AND DIET Chapter...,The context explains that a balanced diet with...,multi_hop_abstract_query_synthesizer
8,how CBT and CBT-I help health and stress,[<1-hop>\n\nPART 2: THERAPEUTIC APPROACHES Cha...,CBT is a therapy that helps change negative th...,multi_hop_specific_query_synthesizer
9,How can understanding the science of habit for...,[<1-hop>\n\nsocial interactions How to set and...,Understanding the science of habit formation f...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [32]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/8 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/18 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [25]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,"As a Mental Health Educator, how would you exp...",[The Mental Health and Psychology Handbook A P...,"Mental health encompasses our emotional, psych...",single_hop_specifc_query_synthesizer
1,What is Mindfulness-Based Stress Reduction and...,[PART 2: THERAPEUTIC APPROACHES Chapter 4: Cog...,Mindfulness-Based Stress Reduction (MBSR) is a...,single_hop_specifc_query_synthesizer
2,"How does serotonin influence mental health, an...",[Write letters to or from your future self Jou...,Serotonin is a neurotransmitter that plays a s...,single_hop_specifc_query_synthesizer
3,"How does social media impact mental health, an...",[social interactions How to set and maintain b...,Social media can significantly impact psycholo...,single_hop_specifc_query_synthesizer
4,how grow mindset help mental health and how it...,[<1-hop>\n\nPART 2: THERAPEUTIC APPROACHES Cha...,The context explains that cultivating a growth...,multi_hop_abstract_query_synthesizer
5,How do nutriens like Omega-3 and B vitamins in...,[<1-hop>\n\nWrite letters to or from your futu...,The context highlights that key nutrients such...,multi_hop_abstract_query_synthesizer
6,how journaling practices and self-awareness he...,[<1-hop>\n\nPART 2: THERAPEUTIC APPROACHES Cha...,"Journaling practices, like writing regularly a...",multi_hop_abstract_query_synthesizer
7,how self-compassion and self-awareness help me...,[<1-hop>\n\nPART 2: THERAPEUTIC APPROACHES Cha...,the context explains that therapy like CBT and...,multi_hop_abstract_query_synthesizer
8,Considering the detailed information provided ...,[<1-hop>\n\nPART 2: NUTRITION AND DIET Chapter...,Understanding the micronutrient requirements o...,multi_hop_specific_query_synthesizer
9,How do Chapters 4 and 20 together inform strat...,[<1-hop>\n\nPART 2: NUTRITION AND DIET Chapter...,Chapter 4 discusses the fundamentals of health...,multi_hop_specific_query_synthesizer


In [33]:
testset.to_pandas().to_csv("./original_rag_testset.csv", index=False)
dataset.to_pandas().to_csv("./original_rag_dataset.csv", index=False)

## ❓ Question #2:

Ragas offers both an "unrolled" (manual) approach and an "abstracted" (automatic) approach to synthetic data generation. What are the trade-offs between these two approaches? When would you choose one over the other?

#### Answer:
First some definitions:

The Unrolled (manual) aproach allows you to explicitly control each step of SDG.
Abstracted (automatic) is where RAGAS will orchestrate all the steps for you.

##### In Unrolled
You manually:
* Select documents
* Choose which query synthesizers to use
* Control distributions (e.g., 50% single-hop, 30% multi-hop, 20% abstract)
* Tune prompts
* Control chunk sampling
* Inspect intermediate outputs

You are “unrolling” the pipeline instead of letting RAGAS hide it behind a high-level function.

###### Pro
* more control
* you can tune to a domain
* you can tune for edge cases and weaknesses
* you control reaonsing depth
* you can tune with real user use cases and traffic patterns
* if it has to be right for a domain like Finance, Legal, Healthcare..

###### Con
* more complex
* you have to understand
  * Query synthesizers
  * Sampling
  * Distribution design
  * Prompt tuning
* more to tune, slower to iterate
* you may get lost in tuning metaparameters and mess things up in there

###### Choose when
* if it has to be right for a domain like Finance, Legal, Healthcare..
* you want to represent real user data closely
* you want to ensure you handle edge, cases and maximize robustness
* you have the time to invest in RAGAS tuning.


##### In Rolled
you call the generator API 
And RAGAS:
* Selects synthesizers
* Balances query types
* Handles orchestration
* Returns a dataset

You don't manage the details of the Pipeline

###### Pro
* less complex
* get running fast for early-stage RAG, hackathons, prototypes..
* RAGAS will use sane deraults

###### Con
* less control
* you dont't know why certain questions were generated..

###### Choose when
* you are short on time
* Its your initial setup, and just want basic test cases
* you don't think there are edge cases
* you don't want to manage a complex system



---
## 🏗️ Activity #1: Custom Query Distribution

Modify the `query_distribution` to experiment with different ratios of query types.

### Requirements:
1. Create a custom query distribution with different weights than the default
2. Generate a new test set using your custom distribution
3. Compare the types of questions generated with the default distribution
4. Explain why you chose the weights you did

## Activity #1 : Answer
I chose to set these weights to be very focused on the Abstract metrics and Multi-hop.  
I was curious how abstract things would get here and how much noise we may introduce with multiple hops and high abstract queries.


To really get a sense I spit out 4 csvs
original_rag_testset.csv
original_rag_dataset.csv

new_rag_testset.csv
new_rag_dataset.csv

I took a look through them, some highlights

#####  original_rag_testset
The single_hop specific questions are truly simpler.  a lot of fact retrieval "What are some mental health resources available online?"

##### new_rag_testset
Since its most all abstract the questions are looking for concepts or impacts, implications etc -> "How does growht mindset impact mental health and implications for emotional well-being?".
There is amost no simple fact recall questions, they are most all asking complex multi-part questions and looking for advice.
Oddly it seems the grammar and spelling are worse here..


In [27]:
### YOUR CODE HERE ###

query_distribution_custom = [
       (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.05),
       (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.05),
       (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.90),
]

testset_new = generator.generate(testset_size=10, query_distribution=query_distribution_custom)
testset_new.to_pandas()



Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,"How does the concept of mental health, as desc...",[The Mental Health and Psychology Handbook A P...,According to the Mental Health and Psychology ...,single_hop_specifc_query_synthesizer
1,How do the concepts discussed in Chapter 4 abo...,[<1-hop>\n\nPART 2: NUTRITION AND DIET Chapter...,Chapter 4 emphasizes the importance of a balan...,multi_hop_specific_query_synthesizer
2,How do dietary patterns and key nutrients such...,[<1-hop>\n\nWrite letters to or from your futu...,The emerging field of nutritional psychiatry h...,multi_hop_abstract_query_synthesizer
3,How can building a workout routine that includ...,[<1-hop>\n\nThe Personal Wellness Guide A Comp...,Building a workout routine that incorporates a...,multi_hop_abstract_query_synthesizer
4,H0w can exercize and movemnt be adivseded acco...,[<1-hop>\n\nThe Personal Wellness Guide A Comp...,The Personal Wellness Guide emphasizes that re...,multi_hop_abstract_query_synthesizer
5,how journaling and self-awareness help mental ...,[<1-hop>\n\nPART 2: THERAPEUTIC APPROACHES Cha...,Journaling helps mental health by allowing sel...,multi_hop_abstract_query_synthesizer
6,how stress management and mental wellness help...,[<1-hop>\n\nPART 2: NUTRITION AND DIET Chapter...,The context explains that stress is body's res...,multi_hop_abstract_query_synthesizer
7,so like what factors influence mental health l...,[<1-hop>\n\nThe Mental Health and Psychology H...,Factors influencing mental health include biol...,multi_hop_abstract_query_synthesizer
8,How does growht mindset impact mental health a...,[<1-hop>\n\nPART 2: THERAPEUTIC APPROACHES Cha...,"The context explains that a growth mindset, wh...",multi_hop_abstract_query_synthesizer
9,How can managing digital wellness and practici...,[<1-hop>\n\n13: The Science of Habit Formation...,Managing digital wellness involves setting spe...,multi_hop_abstract_query_synthesizer


In [28]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset_new = generator.generate_with_langchain_docs(docs, testset_size=10)
dataset_new.to_pandas()

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/7 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/16 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,so like what is the psychology handbook thingy...,[The Mental Health and Psychology Handbook A P...,The Mental Health and Psychology Handbook is a...,single_hop_specifc_query_synthesizer
1,What is Mindfulness-Based Stress Reduction and...,[PART 2: THERAPEUTIC APPROACHES Chapter 4: Cog...,Mindfulness-Based Stress Reduction (MBSR) is a...,single_hop_specifc_query_synthesizer
2,"How does cortisol relate to mental health, esp...",[Write letters to or from your future self Jou...,The provided context discusses how exercise re...,single_hop_specifc_query_synthesizer
3,What does Chapter 16 cover regarding managing ...,[social interactions How to set and maintain b...,Chapter 16: Managing Digital Mental Health dis...,single_hop_specifc_query_synthesizer
4,How exercise and movement like lower back pain...,[<1-hop>\n\nThe Personal Wellness Guide A Comp...,The context explains that regular exercise and...,multi_hop_abstract_query_synthesizer
5,so if im building a workout routine like start...,[<1-hop>\n\nThe Personal Wellness Guide A Comp...,building a workout routine involves starting s...,multi_hop_abstract_query_synthesizer
6,How can habit formation and building a workout...,[<1-hop>\n\n13: The Science of Habit Formation...,Habit formation is essential for establishing ...,multi_hop_abstract_query_synthesizer
7,How does mental health influence physical heal...,[<1-hop>\n\nThe Mental Health and Psychology H...,Mental health significantly impacts physical h...,multi_hop_abstract_query_synthesizer
8,how CBT and CBT-I help mental health and sleep,[<1-hop>\n\nPART 2: THERAPEUTIC APPROACHES Cha...,The context explains that CBT (Cognitive Behav...,multi_hop_specific_query_synthesizer
9,How do Chapters 4 and 20 relate to building he...,[<1-hop>\n\n13: The Science of Habit Formation...,Chapter 4 discusses the fundamentals of health...,multi_hop_specific_query_synthesizer


In [29]:
testset_new.to_pandas().to_csv("./new_rag_testset.csv", index=False)
dataset_new.to_pandas().to_csv("./new_rag_dataset.csv", index=False)

We'll need to provide our LangSmith API key, and set tracing to "true".

---
# 🤝 Breakout Room #2
## RAG Evaluation with LangSmith

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [34]:
from langsmith import Client
import uuid

client = Client()

dataset_name = f"Use Case Synthetic Data - AIE9 - {uuid.uuid4()}"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Synthetic Data for Use Cases"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [35]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [36]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [37]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [38]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [39]:
from langchain_qdrant import QdrantVectorStore

vectorstore = QdrantVectorStore.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="use_case_rag"
)

In [40]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [41]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

As is usual: We'll be using `gpt-4.1-mini` for our RAG!

In [42]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [43]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

#DJY - LCL chain
# conextt pipe to retriever pipe to llm pipe to output parser
rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [44]:
rag_chain.invoke({"question" : "What are some recommended exercises for lower back pain?"})

'Recommended exercises for lower back pain include:\n\n- Cat-Cow Stretch: Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- Partial Crunches: Lie on your back with knees bent, cross arms over chest, tighten stomach muscles and raise shoulders off floor. Hold briefly, then lower. Do 8-12 repetitions.\n- Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n- Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening abs and tilting pelvis up slightly. Hold for 10 seconds, repeat 8-12 times.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [45]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [46]:
from openevals.llm import create_llm_as_judge
from langsmith.evaluation import evaluate

# 1. QA Correctness (replaces LangChainStringEvaluator("qa"))
qa_evaluator = create_llm_as_judge(
    prompt="You are evaluating a QA system. Given the input, assess whether the prediction is correct.\n\nInput: {inputs}\nPrediction: {outputs}\nReference answer: {reference_outputs}\n\nIs the prediction correct? Return 1 if correct, 0 if incorrect.",
    feedback_key="qa",
    model="openai:gpt-4o" ,  # pass your LangChain chat model directly
)

# 2. Labeled Helpfulness (replaces LangChainStringEvaluator("labeled_criteria"))
labeled_helpfulness_evaluator = create_llm_as_judge(
    prompt=(
        "You are assessing a submission based on the following criterion:\n\n"
        "helpfulness: Is this submission helpful to the user, "
        "taking into account the correct reference answer?\n\n"
        "Input: {inputs}\n"
        "Submission: {outputs}\n"
        "Reference answer: {reference_outputs}\n\n"
        "Does the submission meet the criterion? Return 1 if yes, 0 if no."
    ),
    feedback_key="helpfulness",
    model="openai:gpt-4o" ,
)

# 3. Dopeness (replaces LangChainStringEvaluator("criteria"))
dopeness_evaluator = create_llm_as_judge(
    prompt=(
        "You are assessing a submission based on the following criterion:\n\n"
        "dopeness: Is this response dope, lit, cool, or is it just a generic response?\n\n"
        "Input: {inputs}\n"
        "Submission: {outputs}\n\n"
        "Does the submission meet the criterion? Return 1 if yes, 0 if no."
    ),
    feedback_key="dopeness",
    model="openai:gpt-4o" ,
)

> **Describe what each evaluator is evaluating:**
>
> - `qa_evaluator`:
> - `labeled_helpfulness_evaluator`:
> - `dopeness_evaluator`:

## LangSmith Evaluation

In [47]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'vacant-line-13' at:
https://smith.langchain.com/o/af35a50b-6912-4416-9b20-43578813e6ae/datasets/fda62930-58ef-4963-9e3a-e3e5ca0961ef/compare?selectedSessions=aba875b8-3135-4940-8955-15c0ccc38de4




0it [00:00, ?it/s]

KeyboardInterrupt: 

## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [ ]:
DOPENESS_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Make your answer rad, ensure high levels of dopeness. Do not be generic, or give generic responses.

Context: {context}
Question: {question}
"""

dopeness_rag_prompt = ChatPromptTemplate.from_template(DOPENESS_RAG_PROMPT)

In [ ]:
rag_documents = docs

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

## ❓ Question #3:

Why would modifying our chunk size modify the performance of our application?

##### Answer:

Chunk size is one of the highest-leverage knobs in a RAG system because it directly affects:
* What gets embedded
* What gets retrieved
* What gets sent to the LLM
* So changing chunk size changes retrieval quality, context quality, and answer quality.

When you modify chunk size you are changing:
* Vector similarity behavior
* Retrieval ranking
* Multi-hop ability
* Token efficiency
* Signal-to-noise ratio


When an embedding model breaks the document up into chunks, the ideal is to have one concept encapsulated into a chunk, without any noise.  This allows a query to match a chunk very directly and result in cost efficient and accurate retrieval.  If you do multi-hop retrieval then you can hop to multiple chunks and get multiple concepts that match nicely.
If the chunk size is too large, you can get multiple/different concepts in a chunk so the embedding becomes less clear.  A specific question won't match the embedding well as it contains too much stuff that is a distraction.
If chunks are too large then multi-hop systems may be able to get the same answer form one chunk, which could be ok, but presumably you are using multi-hop because you are trying to access multiple different concepts.

If the chunk size too small then each chunk wont contain any real concepts.  A ridiculous example is chunk size ==1 where basically you have a few characters, and no conceptual value to any chunk.  Here no question will truly match any chunk and be useful .
Even if Chunk sizes arent ==1 then small chunks that contain very little info ie just "Happy Birthday", may match queries, but wont provide value.  You'd need so many hops to actually get any value from these small chunks.

From a Context window perspective:

Large chunks:
* Consume more tokens
* Reduce number of chunks you can include
* Increase cost

Small chunks:
* Let you retrieve more disparate pieces
* But risk fragmentation


In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

## ❓ Question #4:

Why would modifying our embedding model modify the performance of our application?

##### Answer:
If 'performance' here relates to the 'goodness' of a result, then different models behave differently.

In a RAG pipeline, embeddings control:
* What gets retrieved
* What gets ranked higher
* What context the LLM sees
* Ultimately, whether the answer is correct

If embeddings change, retrieval changes. If retrieval changes, everything changes.  As goes Retrieval so goes Generation (heard that once or twice :)

Embedding models vary in terms of:
* Semantic understanding
* Domain knowledge
* Training data
* Multilingual ability
* Sensitivity to phrasing

I work at a finance company and would choose an embedding model that understood details like EBITDA, GAAP, cash Flow statements, etc..  this will perform better because it will link these concepts semantically and apply domain knowledge when creating embeddings.  Here it may place COGS semantically close to EBITDA, they don't look alike but COGS is included in the calculation of EBITDA.

Different embeeding models structure their token storage and graph layout differently.  Some are better at storing semantic meaning, others are not going to store as much meaning.
They will all differ in how they place certain chunks "closer" to one another.

Chunk size and embedding model are linked as certain models perform better on longer/shorter text and some handle structured data better.  Back to the finance example, choosing a model that can work with an Excel based balance sheet would be an obvious choice here.

Embedding models also influcences one hop, multi hop.  If an embedding has weak semantic understanding then future hops may not yield benefit and instead may introduce noise.

Some embedding models are higher capacity and capture more semantics, reduce false positives and produce a better overall graph.  Higher capacity embeddings will be slower and more costly.  Lower capacity embeddings will be faster, cheaper, but may blur meaning.  Its a pretty classic quality/performance tradeoff.

If you want faster performance, a smaller embedding model will also be far faster than a complex one, so if 'performance' here is speed, then you have a range of choices.

In [ ]:
from langchain_qdrant import QdrantVectorStore

vectorstore = QdrantVectorStore.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG Docs"
)

In [ ]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [ ]:
dopeness_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dopeness_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [ ]:
dopeness_rag_chain.invoke({"question" : "How can I improve my sleep quality?"})

"Ready to level up your sleep game? Here's your ultimate blueprint for catching those epic Z’s, straight from the sleep sages:\n\n1. **Lock in a consistent sleep schedule**: Hit the sack and rise at the same time every day—even on weekends. Your body’s internal clock thrives on rhythm.\n\n2. **Craft a chill bedtime routine**: Think cozy reading, gentle stretching, or a warm bath—your personal sleep signal to calm down.\n\n3. **Make your bedroom a sleep sanctuary**:\n   - Keep that temp locked between 65-68°F (18-20°C)—cool, but not ice-age.\n   - Blackout curtains or a sleep mask to crush the light.\n   - Embrace quiet vibes—white noise machines or earplugs are your allies.\n   - Invest in a mattress and pillows that feel like cloud nine.\n\n4. **Ditch screens 1-2 hours before bedtime**: Those blue lights hack your melatonin delivery like a sneaky villain.\n\n5. **Cut caffeine after 2 PM**: No late-day jitters sabotaging your slumber.\n\n6. **Exercise regularly, but steer clear of work

Finally, we can evaluate the new chain on the same test set!

In [ ]:
evaluate(
    dopeness_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "dopeness_rag_chain"},
)

View the evaluation results for experiment: 'cold-crate-13' at:
https://smith.langchain.com/o/af35a50b-6912-4416-9b20-43578813e6ae/datasets/aed06739-b74c-4ef3-9438-620ec983bcee/compare?selectedSessions=f5de6905-bc24-45d2-a995-7f1f799f55d7




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.qa,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,H0w do Chapt3er 6 and 8 of the m3ntal health g...,"Yo, let's break it down with some serious brai...",None,Chapt3er 6 of the guide focuses on understandi...,False,False,True,6.333116,c41df1d3-5ff5-425a-80c6-250d05b92d04,019c4a3c-449a-73f0-9fda-e5da06f3bf58
1,"Based on the information in Chapters 3 and 7, ...","Alright, buckle up for a sleep-fueled mind upg...",None,Understanding sleep cycles and sleep hygiene p...,True,True,True,5.779611,15e625af-e093-462b-a9a3-94fbf2e3aac2,019c4a3c-9650-7061-804c-4f7c579747a6
2,Can you tell me about Chapter 4 and Chapter 18...,"Yo, I gotta keep it straight-up fresh and fact...",None,Chapter 4 covers fundamentals of healthy eatin...,False,False,True,2.761874,0ce25403-8669-4d8c-b9be-e409b34d069f,019c4a3c-dad2-70d1-9780-7652e989d742
3,How can I use Chapter 5 and Chapter 21 to impr...,"Yo, let’s fuse Chapter 5’s habit-building magi...",None,To enhance your mental health by building posi...,True,True,True,5.492388,284d3c08-2b28-47df-8e3e-e620ffac0987,019c4a3d-11d8-7c82-96df-7de719650d4e
4,How does understanding the mind-body connectio...,"Alright, buckle up for some next-level mind-bo...",None,The handbook explains that the mind-body conne...,True,True,True,4.420349,4e64742e-8181-403f-a62d-84c14610df6f,019c4a3d-45f8-7910-80d9-b1bbefe4aa1a
5,"What are effective sleep hygiene practices, in...","Alright, here’s the ultimate sleep hack checkl...",None,Effective sleep hygiene practices include main...,True,True,True,7.729413,b1561b25-a2a0-4f97-bb14-af9a7d1a7ee2,019c4a3d-7b31-7f71-8880-3ad95bd7e5f2
6,H0w cna menta health proffesionals uze CBT and...,"Alright, let’s crank the dial up to max dope o...",None,The provided context explains that Cognitive B...,True,True,True,7.598810,890541e4-87c4-4268-9874-7bad1cbe72ee,019c4a3d-ca3d-7552-84ef-aab5ded16b2e
7,How does gut-brain axis and mood regulation co...,"Oh snap, you just unlocked the secret boss lev...",None,The gut-brain axis plays a significant role in...,True,True,True,10.310449,c61dcc1c-9768-4f14-8b40-420dc5c36c46,019c4a3e-0d92-77c1-a80b-7e7d9ac5f875
8,What information does Chapter 1 provide about ...,"Alright, let’s vibe with the wisdom from Chapt...",None,Chapter 1 explains that exercise is one of the...,True,True,True,5.142859,cea5dd5a-e624-4110-af82-5428acaf95b5,019c4a3e-6b3f-7490-b34e-848300b76974
9,What do psychiatrists do?,Psychiatrists are the MVPs of mental health wh...,None,Psychiatrists are medical doctors who can pres...,True,True,True,1.917401,9c20a26f-3466-4c46-b859-6dd2c1c61039,019c4a3e-98c5-7f82-9c8b-bb6cf1057579


---
## 🏗️ Activity #2: Analyze Evaluation Results

Provide a screenshot of the difference between the two chains in LangSmith, and explain why you believe certain metrics changed in certain ways.

##### Answer:
In my dataset 'cloud-crate-13' is the run that included the 'dopeness' prompt and 'proper-popcorn-25 is not'
Given we have a direct measure for 'dopeness' and  the cloud-crate prompt was explicitly told to be 'dope' AND the evaluation framework is assessing 'dopeness' then that will match.  The initial prompting was not told to be dope, so its being evaluated on something it wasn't really trained to do.  So its a pretty unfair analysis.

For the Comparison Dashboard screenshot
We can see the cloud crate has dopeness == 1.0 and popcorn == 0.25 so way lower.
Also on the helpfulness and qa rows the 'dope' cloud-crate did way better.  Again the evaluator was looking for more dopeness, and that dataset delivered.

For each individual message screenshot we can see that cloud-crate message by message ws viewed to be doper, more helpful and more 'correct'.  Of course the non dope LLM responded with 'i dont know' a massive 5 of 12 times, which really hurts it across the board.  Again more and more runs here would be needed to truly see.

The dopeness run seemed to be slower for requests on average, perhaps due to the extra prompting to be 'dope'.  I'd run them each about 10 times and really get a timing to determine if being dope == being slower, one run isnt statistically signifcant here.

From a token perspective
'dope' - 3k output tokens, e
'not dope' - a bunch of 'i dont know' responses, really cut down the token count.  the input tokens were slightly higher.

filtering out the 'i dont know responses' the average tokens for outputs is higher for dopeness responses, because we're inserting un-needed stuff like 'ok dude ' and etc.. to make it dope.

the original run used these parameters
* embedding = `text-embedding-3-small`
* chunk size = 500
* chunk overlap = 50

Dopeness run
* embedding = `text-embedding-3-large`
* chunk size = 1000
* chunk overlap = 50

so we upgraded our embedding model AND went to larger chunk size on the dopeness run.

The fact this led us to consume fewer INPUT tokens on the dopeness runs is strange..  I WOULD THINK with a larger chunk size and more advanced embedding model and adding all that extra fluff language to be 'dope' would have driven the token count up 


## LangSmith Evaluation Results

### Experiment Comparison Dashboard
![Experiment Comparison](./Screen%20Shot%202026-02-16%20at%2011.48.50%20AM.png)

### Experiment: cold-crate-13 (Detailed Results)
![cold-crate-13 Results](./Screen%20Shot%202026-02-16%20at%2011.49.10%20AM.png)

### Experiment: proper-popcorn-25 (Detailed Results)
![proper-popcorn-25 Results](./Screen%20Shot%202026-02-16%20at%2011.49.24%20AM.png)


---
## Summary

In this session, we:

1. **Generated synthetic test data** using Ragas' knowledge graph-based approach
2. **Explored query synthesizers** for creating diverse question types
3. **Loaded synthetic data** into a LangSmith dataset for evaluation
4. **Built and evaluated a RAG chain** using LangSmith evaluators
5. **Iterated on the pipeline** by modifying chunk size, embedding model, and prompt — then measured the impact

### Key Takeaways:

- **Synthetic data generation** is critical for early iteration — it provides high-quality signal without manually creating test data
- **LangSmith evaluators** enable systematic comparison of pipeline versions
- **Small changes matter** — chunk size, embedding model, and prompt modifications can significantly affect evaluation scores